# exploring 2017 test and training sets

In [196]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
train = pd.read_csv('BPI Challenge 2017-training.csv')
test = pd.read_csv('BPI Challenge 2017-test.csv')

In [3]:
test_time = test['event time:timestamp'].copy()
train_time = train['event time:timestamp'].copy()

In [4]:
train.head()

,eventID,case LoanGoal,case ApplicationType,case concept:name,case RequestedAmount,event Action,event org:resource,event concept:name,event EventOrigin,event EventID,event lifecycle:transition,event time:timestamp
0,0,Existing loan takeover,New credit,Application_652823628,20000.0,Created,User_1,A_Create Application,Application,Application_652823628,complete,01-01-2016 10:51:15.304
1,1,Existing loan takeover,New credit,Application_652823628,20000.0,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,01-01-2016 10:51:15.352
2,2,Existing loan takeover,New credit,Application_652823628,20000.0,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,01-01-2016 10:51:15.774
3,3,Existing loan takeover,New credit,Application_652823628,20000.0,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,01-01-2016 10:52:36.392
4,4,Existing loan takeover,New credit,Application_652823628,20000.0,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,01-01-2016 10:52:36.403


In [5]:
test.head()

,eventID,case LoanGoal,case ApplicationType,case concept:name,case RequestedAmount,event Action,event org:resource,event concept:name,event EventOrigin,event EventID,event lifecycle:transition,event time:timestamp
0,108263240630272,Existing loan takeover,New credit,Application_424064154,17250.0,Created,User_1,A_Create Application,Application,Application_424064154,complete,19-10-2016 01:42:39.265
1,108263240630273,Existing loan takeover,New credit,Application_424064154,17250.0,statechange,User_1,A_Submitted,Application,ApplState_875814280,complete,19-10-2016 01:42:40.100
2,108263240630274,Existing loan takeover,New credit,Application_424064154,17250.0,Created,User_1,W_Handle leads,Workflow,Workitem_1964338518,schedule,19-10-2016 01:42:40.711
3,108267535597568,"Other, see explanation",New credit,Application_608775916,10000.0,Created,User_1,A_Create Application,Application,Application_608775916,complete,19-10-2016 04:16:17.680
4,108267535597569,"Other, see explanation",New credit,Application_608775916,10000.0,statechange,User_1,A_Submitted,Application,ApplState_1860095003,complete,19-10-2016 04:16:18.446


In [6]:
#dt_test = pd.to_datetime(test_time)
#dt_train = pd.to_datetime(train_time)

In [7]:
# dt_test

In [8]:
# dt_train

# Set 2012

In [150]:
train = pd.read_csv('BPI_Challenge_2012-training.csv')
test = pd.read_csv('BPI_Challenge_2012-test.csv')

In [3]:
train.head()

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256


In [4]:
train.rename(columns={'case concept:name':'case_name'}, inplace=True)

In [12]:
cases = list(train.case_name.unique())

In [13]:
# dicts={}

# for case in cases:
    
#     ran = len(train[train['case_name']==case])
#     ID = train[['event concept:name']][train['case_name']==case]
    
#     for i in range(ran):
#         dicts[i]= ID.iloc[i][0]

# Method github

In [5]:
df_sort = train.set_index(train.groupby('case_name').cumcount(), append = True)

In [6]:
df_new = df_sort.groupby(level=1)["event concept:name"].transform(lambda x: x.value_counts().index[0])
df_new = df_new.to_frame()
df_new = df_new.rename(columns = {'event concept:name' : "common activity"})
df_sort = df_sort.join(df_new["common activity"])

# Explore time differences

In [7]:
df_sort = df_sort.rename(columns = {'event time:timestamp' : "timestamp"})
df_sort['timestamp'] = pd.to_datetime(df_sort.timestamp)

In [8]:
df_times = df_sort.groupby('case_name')["timestamp"].transform(lambda x: x.diff())
df_times = df_times.to_frame()
df_times = df_times.rename(columns = {'timestamp' : "time_diff"})

In [9]:
df_sort = df_sort.join(df_times["time_diff"])

In [16]:
df_sort[df_sort['case_name']==173688].head()

,,eventID,case_name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,timestamp,common activity,time_diff
0,0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,2011-01-10 00:38:44.546,A_SUBMITTED,NaT
1,1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,2011-01-10 00:38:44.880,A_PARTLYSUBMITTED,00:00:00.334000
2,2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,2011-01-10 00:39:37.906,W_Afhandelen leads,00:00:53.026000
3,3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,2011-01-10 00:39:38.875,W_Afhandelen leads,00:00:00.969000
89,4,4,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,START,2011-01-10 11:36:46.437,W_Completeren aanvraag,10:57:07.562000


# Function for adding the next event

In [291]:
def make_event_pred(df_train, df_test):
    
    '''Parameters: df_train: training set
                   df_test: test set
                   
       We can change to csv as input and making the sets or create a different function for it'''
    preds = pd.DataFrame() # we can return this df in order to et only the predictions for accuracy scores
    
    df_sort = df_train.set_index(df_train.groupby('case concept:name').cumcount(), append = True)
    df_new = df_sort.groupby(level=1)["event concept:name"].transform(lambda x: x.value_counts().index[0])
    df_new = df_new.to_frame()
    common_acts = df_new.rename(columns = {'event concept:name' : "common activity"})
    
    test = df_test.set_index(df_test.groupby('case concept:name').cumcount(), append = True)
    
    # Here we can implement the average time to add later in the dataframe
    test = test.rename(columns = {'event time:timestamp' : "timestamp"})
    test['timestamp'] = pd.to_datetime(test.timestamp) 
    
    
    max_list = common_acts.groupby(level=1).apply(max).copy() # Get process with max(N) for predictions
    
    for i, df in test.groupby('case concept:name'):
        length = len(df)
        test = test.append({'case concept:name':int(i),
                            'event concept:name': max_list.loc[length, 'common activity']}, ignore_index=True)
                            #'timestamp': dt.datetime(2012, 2, 14, 23, 59, 59)}, ignore_index=True)
                            # here we can add the time of the added new event
                
        preds = preds.append({'case concept:name':int(i),
                            'event concept:name': max_list.loc[length, 'common activity']}, ignore_index=True)
                            #'timestamp': dt.datetime(2012, 2, 14, 23, 59, 59)}, ignore_index=True) 
            
    test = test.set_index(test.groupby('case concept:name').cumcount(), append = True)
    
    # Depending on how we want it we should sort: test = test.sort_values(time, case name, etc.)
    test = test.sort_values('timestamp') # now sorted by time
    
    return preds #test

# Test stuff for the function

In [148]:
pos = df_new.groupby(level=1).apply(max).copy()

In [214]:
df_test = test.set_index(test.groupby('case concept:name').cumcount(), append = True)
df_test.tail()

,,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
47818,35,54666343743523,213276,2012-02-27T14:12:41.868+01:00,15000,W_Nabellen incomplete dossiers,START,14-03-2012 15:59:28.309
47819,36,54666343743524,213276,2012-02-27T14:12:41.868+01:00,15000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 16:00:09.680
47820,32,49495203119136,209595,2012-02-15T10:10:36.503+01:00,13000,W_Nabellen offertes,START,14-03-2012 16:02:03.883
47821,34,52342766436386,211624,2012-02-21T23:38:40.044+01:00,35000,W_Nabellen incomplete dossiers,START,14-03-2012 16:04:46.192
47822,35,52342766436387,211624,2012-02-21T23:38:40.044+01:00,35000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 16:04:54.681


In [215]:
df_test = df_test.rename(columns = {'event time:timestamp' : "timestamp"})
df_test['timestamp'] = pd.to_datetime(df_test.timestamp)

for i, df in df_test.groupby('case concept:name'):
    length = len(df)
    df_test = df_test.append({'case concept:name':int(i),
                              'event concept:name': pos.loc[length, 'common activity'],
                              'timestamp': dt.datetime(2012, 12, 30, 23, 59, 59)},ignore_index=True)
df_test = df_test.set_index(df_test.groupby('case concept:name').cumcount(), append = True)

In [216]:
df_test[df_test['case concept:name']==206324]

,,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,timestamp
0,0,4.496401e+13,206324,2012-02-03T17:17:11.047+01:00,2500.0,A_SUBMITTED,COMPLETE,2012-03-02 17:17:11.047
1,1,4.496401e+13,206324,2012-02-03T17:17:11.047+01:00,2500.0,A_PARTLYSUBMITTED,COMPLETE,2012-03-02 17:17:11.323
2,2,4.496401e+13,206324,2012-02-03T17:17:11.047+01:00,2500.0,A_DECLINED,COMPLETE,2012-03-02 17:17:42.964
47823,3,NaN,206324,NaN,NaN,W_Afhandelen leads,NaN,2010-12-30 23:59:59.000


In [219]:
df_test.sort_values('timestamp')

# Testing function

In [292]:
train = pd.read_csv('BPI_Challenge_2012-training.csv')
test = pd.read_csv('BPI_Challenge_2012-test.csv')

In [293]:
saved = make_event_pred(train, test)

In [294]:
saved[saved['case concept:name']==206327]

,case concept:name,event concept:name
1,206327.0,W_Completeren aanvraag


In [295]:
saved

,case concept:name,event concept:name
0,206324.0,W_Afhandelen leads
1,206327.0,W_Completeren aanvraag
2,206330.0,W_Afhandelen leads
3,206333.0,W_Nabellen incomplete dossiers
4,206336.0,W_Nabellen incomplete dossiers
...,...,...
2613,214364.0,W_Nabellen offertes
2614,214367.0,W_Afhandelen leads
2615,214370.0,W_Completeren aanvraag
2616,214373.0,W_Nabellen offertes
